In [1]:
import numpy as np
import sympy
import hls4ml

/afs/cern.ch/work/h/htsoi/miniconda3/envs/sr/lib/python3.9/site-packages/hls4ml/converters/__init__.py:16: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!")


# Pass trained expressions to hls4ml converter and compile

In [2]:
expr_str = ['x0+x1',
            '(x0+x1)**2',
            'exp(x2+x3)',
            'x3**3',
            'x4*x5*x5']
expr = []
for i in range(5):
    expr.append(sympy.parsing.sympy_parser.parse_expr(expr_str[i]))

In [3]:
hls_model = hls4ml.converters.convert_from_symbolic_expression(expr,
                                                               n_symbols=6,
                                                               output_dir='my-hls-test',
                                                               precision='ap_fixed<16, 10>')

hls_model.write()
hls_model.compile()

Writing HLS project
Done
Writing HLS project
Done


In [4]:
hls_model.predict(np.array([1,2,3,4,5,6], dtype=np.float32))

array([  3.   ,   4.   ,  72.625,  64.   , 180.   ], dtype=float32)

In [5]:
!cat my-hls-test/firmware/myproject.cpp 

//
//    rfnoc-hls-neuralnet: Vivado HLS code for neural-net building blocks
//
//    Copyright (C) 2017 EJ Kreinar
//
//    This program is free software: you can redistribute it and/or modify
//    it under the terms of the GNU General Public License as published by
//    the Free Software Foundation, either version 3 of the License, or
//    (at your option) any later version.
//
//    This program is distributed in the hope that it will be useful,
//    but WITHOUT ANY WARRANTY; without even the implied warranty of
//    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
//    GNU General Public License for more details.
//
//    You should have received a copy of the GNU General Public License
//    along with this program.  If not, see <http://www.gnu.org/licenses/>.
//
#include <iostream>

#include "myproject.h"
#include "parameters.h"

void myproject(
    input_t x[N_INPUT_1_1],
    result_t y[N_OUTPUTS_2]
) {

    //hls-fpga-machine-learning insert IO
    #pragma HL

# Compare PySR and hls4ml accuracy

# Run synthesis from command line

In [ ]:
source /data/tools/Xilinx/Vivado/2020.1/settings64.sh
source /data/setup_xilinx_lic.sh

In [ ]:
!vivado_hls -f build_prj.tcl "reset=1 synth=1 csim=0 cosim=0 validation=0 export=0 vsynth=0"

In [6]:
#hls4ml.report.read_vivado_report('my-hls-test/')
!cat my-hls-test/myproject_prj/solution1/syn/report/myproject_csynth.rpt



== Vivado HLS Report for 'myproject'
* Date:           Sat Nov  5 08:32:23 2022

* Version:        2020.1 (Build 2897737 on Wed May 27 20:21:37 MDT 2020)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu9p-flga2577-2-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+---------+----------+------------+
    |  Clock |  Target | Estimated| Uncertainty|
    +--------+---------+----------+------------+
    |ap_clk  | 5.00 ns | 4.094 ns |   0.62 ns  |
    +--------+---------+----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+-----------+-----------+-----+-----+----------+
    |  Latency (cycles) |   Latency (absolute)  |  Interval | Pipeline |
    |   min   |   max   |    min    |    max    | min | max |   Type   |
    +---------+---------+-----------+-----------+-----+-----+----------+
    |        2|        2| 10.000 ns | 10.000 ns |    1|    1| function |
    +---------+--------